In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib
from numpy import random
import copy
import scipy

In [ ]:
# for fonts of the plots
matplotlib.rc("font", **{"family":"serif","serif":["Computer Modern Roman"]})
matplotlib.rc("text", usetex=True)

## Non-interacting self-circling simulation data

In [ ]:
# simulation parameters

numx = 30
numy = 30
N = numx*numy
R = 0.5
spacing = 1.0
dx = 0.3*spacing
dy = 0.3*spacing
Lx = spacing * 2 * R * numx
Ly = 0.5 * np.sqrt(3) * numy * spacing * 2 * R
xnum = round(Lx/dx)
ynum = round(Ly/dy)
nrec = 1000
n_j = int(nrec*0.8)
wnum = int(n_j/2)+1
dkx = 2*np.pi/Lx
dky = 2*np.pi/Ly
dt = 0.4
dw = 2*np.pi/(dt*n_j)

# frequency domain 

kxrange = np.zeros(xnum)
kyrange = np.zeros(ynum)
wrange = dw*np.arange(wnum)

for i in range(xnum):
    kxrange[i] = -np.pi/dx + dkx*i
for j in range(ynum):
    kyrange[j] = -np.pi/dy + dky*j
    
kx,ky = np.meshgrid(kxrange,kyrange,indexing='ij')

# load current correlation files

with open('Cll_nisc.dat','rb') as f:
    Cll = np.fromfile(f,np.float64)
Cll = (1/N)*Cll.reshape((xnum,ynum,wnum))

with open('Clt_r_nisc.dat','rb') as f:
    Cltr = np.fromfile(f,np.float64)
Cltr = (1/N)*Cltr.reshape((xnum,ynum,wnum))

with open('Clt_i_nisc.dat','rb') as f:
    Clti = np.fromfile(f,np.float64)
Clti = (1/N)*Clti.reshape((xnum,ynum,wnum))

with open('Ctt_nisc.dat','rb') as f:
    Ctt = np.fromfile(f,np.float64)
Ctt = (1/N)*Ctt.reshape((xnum,ynum,wnum))

Cfull = Cll + Ctt


## Dispersion analysis

In [ ]:
# 2d matrix whose elements are the max value of C at particular (qx,qy)
# preserve the sign of Cltr and Clti

Cll_copy = copy.copy(Cll)
Cltr_copy = copy.copy(Cltr)
Clti_copy = copy.copy(Clti)
Ctt_copy = copy.copy(Ctt)
Cfull_copy = copy.copy(Cfull)

Cll_peak = np.zeros([xnum,ynum])
Cltr_peak = np.zeros([xnum,ynum])
Clti_peak = np.zeros([xnum,ynum])
Ctt_peak = np.zeros([xnum,ynum])

Cfull_peak = np.zeros([xnum,ynum])

wll_peak = np.zeros([xnum,ynum])
wltr_peak = np.zeros([xnum,ynum])
wlti_peak = np.zeros([xnum,ynum])
wtt_peak = np.zeros([xnum,ynum])

wfull_peak = np.zeros([xnum,ynum])

fromhere = 1

for i in range(xnum):
    for j in range(ynum):
        Cll_thisq = Cll_copy[i,j,fromhere:]
        Ctt_thisq = Ctt_copy[i,j,fromhere:]
        Cltr_thisq = Cltr_copy[i,j,fromhere:]
        Clti_thisq = Clti_copy[i,j,fromhere:]
        Cltr_thisq_pos = abs(Cltr_copy[i,j,fromhere:])
        Clti_thisq_pos = abs(Clti_copy[i,j,fromhere:])
        Cfull_thisq = Cfull_copy[i,j,fromhere:]
        
        Cll_peak[i,j] = np.max(Cll_thisq) # record max C value at certain q
        idx_wllmax = np.where(Cll_thisq==np.max(Cll_thisq)) # index in w dimension for the max C
        idx_wllmax = int(idx_wllmax[0][0]) + fromhere # make the index an integer and add fromhere
        wll_peak[i,j] = wrange[idx_wllmax] # record the value of max peak w. 
        
        Ctt_peak[i,j] = np.max(Ctt_thisq) # record max C value at certain q
        idx_wttmax = np.where(Ctt_thisq==np.max(Ctt_thisq)) # index in w dimension for the max C
        idx_wttmax = int(idx_wttmax[0][0]) + fromhere # make the index an integer and add fromhere
        wtt_peak[i,j] = wrange[idx_wttmax] # record the value of max peak w. 
        
        idx_wltrmax = np.where(Cltr_thisq_pos==np.max(Cltr_thisq_pos))
        idx_wltrmax = int(idx_wltrmax[0][0]) + fromhere
        wltr_peak[i,j] = wrange[idx_wltrmax]
        Cltr_peak[i,j] = Cltr_thisq[idx_wltrmax - fromhere]
        
        idx_wltimax = np.where(Clti_thisq_pos==np.max(Clti_thisq_pos))
        idx_wltimax = int(idx_wltimax[0][0]) + fromhere
        wlti_peak[i,j] = wrange[idx_wltimax]
        Clti_peak[i,j] = Clti_thisq[idx_wltimax - fromhere]
        
        Cfull_peak[i,j] = np.max(Cfull_thisq)
        idx_wfullmax = np.where(Cfull_thisq==np.max(Cfull_thisq))
        idx_wfullmax = int(idx_wfullmax[0][0]) + fromhere
        wfull_peak[i,j] = wrange[idx_wfullmax]
        

## Generate the figure of the dispersion result

In [ ]:
# points for reciprocal lattice and the first BZ

p1x = 0.0 # kx value of the vertex of reciprocal lattice obtained from previous result
p1y = 4*np.pi/(np.sqrt(3)*spacing) # ky value of the vertex of reciprocal lattice obtained from previous result

p2x = (1/2)*p1x-(np.sqrt(3)/2)*p1y
p2y = (np.sqrt(3)/2)*p1x+(1/2)*p1y

p3x = (1/2)*p2x-(np.sqrt(3)/2)*p2y
p3y = (np.sqrt(3)/2)*p2x+(1/2)*p2y

p4x = (1/2)*p3x-(np.sqrt(3)/2)*p3y
p4y = (np.sqrt(3)/2)*p3x+(1/2)*p3y

p5x = (1/2)*p4x-(np.sqrt(3)/2)*p4y
p5y = (np.sqrt(3)/2)*p4x+(1/2)*p4y

p6x = (1/2)*p5x-(np.sqrt(3)/2)*p5y
p6y = (np.sqrt(3)/2)*p5x+(1/2)*p5y

bz1x = (1/2)*p1x + (1/(2*np.sqrt(3)))*p1y
bz1y = -(1/(2*np.sqrt(3)))*p1x + (1/2)*p1y

bz2x = (1/2)*bz1x-(np.sqrt(3)/2)*bz1y
bz2y = (np.sqrt(3)/2)*bz1x+(1/2)*bz1y

bz3x = (1/2)*bz2x-(np.sqrt(3)/2)*bz2y
bz3y = (np.sqrt(3)/2)*bz2x+(1/2)*bz2y

bz4x = (1/2)*bz3x-(np.sqrt(3)/2)*bz3y
bz4y = (np.sqrt(3)/2)*bz3x+(1/2)*bz3y

bz5x = (1/2)*bz4x-(np.sqrt(3)/2)*bz4y
bz5y = (np.sqrt(3)/2)*bz4x+(1/2)*bz4y

bz6x = (1/2)*bz5x-(np.sqrt(3)/2)*bz5y
bz6y = (np.sqrt(3)/2)*bz5x+(1/2)*bz5y


In [ ]:
C_plot = copy.copy(Cfull_peak)

C_qx0 = copy.copy(Cfull)
C_qx0 = C_qx0[50,:,:] # slice at qx = 0
q_slice,w_slice = np.meshgrid(kyrange,wrange,indexing='ij')
fromthisw = 1 # not include w=0
uptothisw = -1 # high frequency w is not relevant
w_slice = w_slice[:,fromthisw:uptothisw] # only take w>0.
q_slice = q_slice[:,fromthisw:uptothisw]
C_slice = C_qx0[:,fromthisw:uptothisw]

%matplotlib inline

fig = plt.figure(facecolor=(1,1,1),figsize=(25,9))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

# first plot
img1 = ax1.contourf(kx,ky,C_plot,levels=100)

ax1.plot([p1x,p2x,p3x,p4x,p5x,p6x,p1x],[p1y,p2y,p3y,p4y,p5y,p6y,p1y],'k')
ax1.plot([bz1x,bz2x,bz3x,bz4x,bz5x,bz6x,bz1x],[bz1y,bz2y,bz3y,bz4y,bz5y,bz6y,bz1y],'k')

ax1.plot([0,0],[-10,10],'r--')

cb1 = fig.colorbar(img1,ax=ax1)
ax1.set_xlabel('$q_x$',fontsize=30)
ax1.set_ylabel('$q_y$',fontsize=30)
cb1.set_label('C',fontsize=30)
ax1.text(-13,10.2,'(a)',fontsize=30,va='top',ha='right')
ax1.tick_params(labelsize=20)
cb1.ax.tick_params(labelsize=20)
#plt.gca().set_aspect('equal')

# second plot
img2 = ax2.contourf(q_slice[:,:100],w_slice[:,:100],C_slice[:,:100],levels=100)

cb2 = fig.colorbar(img2,ax=ax2)
ax2.set_xlabel('$q_y$',fontsize=30)
ax2.set_ylabel('$\omega$',fontsize=30)
cb2.set_label('C',fontsize=30)
ax2.tick_params(labelsize=20)
cb2.ax.tick_params(labelsize=20)
ax2.set_box_aspect(aspect=1)
ax2.text(-12.5,1.95,'(b)',fontsize=30,va='top',ha='right')

# to avoid the weird white lines appearing in pdf file of the plot
for c in img1.collections:
    c.set_edgecolor("face")

for c in img2.collections:
    c.set_edgecolor("face")

#plt.savefig('simple_circling_v3.pdf',dpi=200,bbox_inches='tight')
plt.show()
